In [63]:
import lightgbm as lgb
import sklearn

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc,accuracy_score,roc_auc_score
from sklearn.model_selection import StratifiedKFold
# Onehot encoding

from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy import sparse
import gc
import os
import random

In [40]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    #torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False
seed_everything()

In [19]:
!mkdir input/
%cd input/
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
!unzip NewsAggregatorDataset.zip



mkdir: cannot create directory ‘input/’: File exists
/lib/modules/input
--2021-12-12 09:05:35--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip.1’

 NewsAggregatorData  19%[==>                 ]   5.32M  2.90MB/s               ^C
Archive:  NewsAggregatorDataset.zip
replace 2pageSessions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [13]:
# 読込時のエラー回避のためダブルクォーテーションをシングルクォーテーションに置換
!sed -e 's/"/'\''/g' ./input/newsCorpora.csv > ./input/newsCorpora_re.csv
#%cd ..

In [14]:


# データの読込
df = pd.read_csv('./input/newsCorpora_re.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# データの抽出
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]



In [15]:
df.head()

,TITLE,CATEGORY
12,Europe reaches crunch point on banking union,b
13,ECB FOCUS-Stronger euro drowns out ECB's messa...,b
19,"Euro Anxieties Wane as Bunds Top Treasuries, S...",b
20,Noyer Says Strong Euro Creates Unwarranted Eco...,b
29,REFILE-Bad loan triggers key feature in ECB ba...,b


In [16]:
# データの分割
df_train, df_valid_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=123, stratify=df['CATEGORY'])
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, shuffle=True, random_state=123, stratify=df_valid_test['CATEGORY'])
df_train.reset_index(drop=True, inplace=True)
df_valid.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

print(df_train.head())

                                               TITLE CATEGORY
0  REFILE-UPDATE 1-European car sales up for sixt...        b
1  Amazon Plans to Fight FTC Over Mobile-App Purc...        t
2  Kids Still Get Codeine In Emergency Rooms Desp...        m
3  What On Earth Happened Between Solange And Jay...        e
4  NATO Missile Defense Is Flight Tested Over Hawaii        b


In [22]:
vect_word = TfidfVectorizer(max_features=20000, lowercase=True, analyzer='word',
                        stop_words= 'english',ngram_range=(1,3),dtype=np.float32)
vect_char = TfidfVectorizer(max_features=40000, lowercase=True, analyzer='char',
                        stop_words= 'english',ngram_range=(3,6),dtype=np.float32)

In [28]:
# Word ngram vector
tr_vect = vect_word.fit_transform(df_train['TITLE'])
vl_vect = vect_word.transform(df_valid['TITLE'])
ts_vect = vect_word.transform(df_test['TITLE'])

# Character n gram vector
tr_vect_char = vect_char.fit_transform(df_train['TITLE'])
vl_vect_char = vect_char.transform(df_valid['TITLE'])
ts_vect_char = vect_char.transform(df_test['TITLE'])
gc.collect()

16

In [34]:
tr_vect.shape

(10684, 20000)

In [29]:
X = sparse.hstack([tr_vect, tr_vect_char])
x_val = sparse.hstack([vl_vect, vl_vect_char])
x_test = sparse.hstack([ts_vect, ts_vect_char])

In [57]:
le = LabelEncoder()
y_tr = le.fit_transform(df_train['CATEGORY'].values)
y_vl = le.transform(df_valid['CATEGORY'].values)
y_te = le.transform(df_test['CATEGORY'].values)

In [42]:
svd = TruncatedSVD(n_components=300, random_state=42)
X = svd.fit_transform(tr_vect)
x_val = svd.transform(vl_vect)
x_test = svd.transform(ts_vect)

In [43]:
X.shape

(10684, 300)

In [44]:
y_vl.shape

(1336, 4)

In [49]:
y_tr.shape

(10684, 4)

In [58]:
model = lgb.LGBMClassifier()
model.fit(X, y_tr)

LGBMClassifier()

In [68]:
# 検証データを予測する
y_pred = model.predict_proba(x_val)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

accuracy = sum(y_vl == y_pred_max) / len(y_vl)
print(accuracy)

0.8787425149700598


In [69]:
print(roc_auc_score(y_vl, y_pred, multi_class='ovo'))
print(log_loss(y_vl, y_pred))

0.9514706810553059
0.3888652444277943


In [70]:
# 評価データを予測する
y_pred = model.predict_proba(x_test)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

accuracy = sum(y_te == y_pred_max) / len(y_te)
print(accuracy)

0.8839820359281437


In [71]:
print(roc_auc_score(y_te, y_pred, multi_class='ovo'))
print(log_loss(y_te, y_pred))

0.9547788083432888
0.36538368891460654


# Gensim

In [72]:
!pip freeze > requirements.lock

In [74]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [90]:
class SWEM():
    """
    Simple Word-Embeddingbased Models (SWEM)
    https://arxiv.org/abs/1805.09843v1
    """

    def __init__(self, w2v, tokenizer, oov_initialize_range=(-0.01, 0.01)):
        self.w2v = w2v
        self.tokenizer = tokenizer
        self.vocab = set(self.w2v.vocab.keys())
        self.embedding_dim = self.w2v.vector_size
        self.oov_initialize_range = oov_initialize_range

        if self.oov_initialize_range[0] > self.oov_initialize_range[1]:
            raise ValueError("Specify valid initialize range: "
                             f"[{self.oov_initialize_range[0]}, {self.oov_initialize_range[1]}]")

    def get_word_embeddings(self, text):
        np.random.seed(abs(hash(text)) % (10 ** 8))

        vectors = []
        for word in self.tokenizer(text):
            if word in self.vocab:
                vectors.append(self.w2v[word])
            else:
                vectors.append(np.random.uniform(self.oov_initialize_range[0],
                                                 self.oov_initialize_range[1],
                                                 self.embedding_dim))
        return np.array(vectors)

    def average_pooling(self, text):
        word_embeddings = self.get_word_embeddings(text)
        return np.mean(word_embeddings, axis=0)

    def max_pooling(self, text):
        word_embeddings = self.get_word_embeddings(text)
        return np.max(word_embeddings, axis=0)

    def concat_average_max_pooling(self, text):
        word_embeddings = self.get_word_embeddings(text)
        return np.r_[np.mean(word_embeddings, axis=0), np.max(word_embeddings, axis=0)]

    def hierarchical_pooling(self, text, n):
        word_embeddings = self.get_word_embeddings(text)

        text_len = word_embeddings.shape[0]
        if n > text_len:
            raise ValueError(f"window size must be less than text length / window_size:{n} text_length:{text_len}")
        window_average_pooling_vec = [np.mean(word_embeddings[i:i + n], axis=0) for i in range(text_len - n + 1)]

        return np.max(window_average_pooling_vec, axis=0)

In [91]:
from gensim.models import KeyedVectors

from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()

swem = SWEM(wv, nlp.tokenizer)

In [114]:
# Word ngram vector
tr_vect = np.array([swem.average_pooling(text) for text in df_train['TITLE'].tolist()])
vl_vect = np.array([swem.average_pooling(text) for text in df_valid['TITLE'].tolist()])
ts_vect = np.array([swem.average_pooling(text) for text in df_test['TITLE'].tolist()])

In [107]:
tr_vect.shape

(10684, 300)

In [108]:
y_tr.shape

(10684,)

In [112]:
pd.Series(y_tr).value_counts()

0    4501
1    4235
3    1220
2     728
dtype: int64

In [110]:
model = lgb.LGBMClassifier()
model.fit(tr_vect, y_tr)

LGBMClassifier()

In [115]:
# 検証データを予測する
y_pred = model.predict_proba(vl_vect)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

accuracy = sum(y_vl == y_pred_max) / len(y_vl)
print('{:.4f}'.format(accuracy))
print('{:.4f}'.format(roc_auc_score(y_vl, y_pred, multi_class='ovo')))
print('{:.4f}'.format(log_loss(y_vl, y_pred)))

0.4835
0.5508
1.1765


In [117]:
# 評価データを予測する
y_pred = model.predict_proba(ts_vect)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

accuracy = sum(y_te == y_pred_max) / len(y_te)
print('{:.4f}'.format(accuracy))
print('{:.4f}'.format(roc_auc_score(y_te, y_pred, multi_class='ovo')))
print('{:.4f}'.format(log_loss(y_te, y_pred)))

0.5037
0.5648
1.1567


# GloVe

In [118]:
# GloVeダウンロード
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2021-12-12 10:41:52--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-12-12 10:41:53--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  4.29MB/s    in 3m 31s  

2021-12-12 10:45:24 (3.91 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: 

In [119]:
EMBEDDING_FILE='./glove.6B.300d.txt'

In [120]:
# Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [124]:
from keras.preprocessing.text import Tokenizer

2021-12-12 14:43:01.260372: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-12 14:43:01.262445: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [125]:
embedding_dict={}
with open(EMBEDDING_FILE,'r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [142]:
class SWEM_Glove():
    """
    Simple Word-Embeddingbased Models (SWEM)
    https://arxiv.org/abs/1805.09843v1
    """

    def __init__(self, dic, tokenizer, oov_initialize_range=(-0.01, 0.01)):
        self.tokenizer = tokenizer
        self.dic = dic
        self.embedding_dim = self.dic['a'].shape[0]
        self.oov_initialize_range = oov_initialize_range

        if self.oov_initialize_range[0] > self.oov_initialize_range[1]:
            raise ValueError("Specify valid initialize range: "
                             f"[{self.oov_initialize_range[0]}, {self.oov_initialize_range[1]}]")

    def get_word_embeddings(self, text):
        np.random.seed(abs(hash(text)) % (10 ** 8))

        vectors = []
        for word in text.split():
            if word in self.dic:
                vectors.append(self.dic[word])
            else:
                vectors.append(np.random.uniform(self.oov_initialize_range[0],
                                                 self.oov_initialize_range[1],
                                                 self.embedding_dim))
        return np.array(vectors)

    def average_pooling(self, text):
        word_embeddings = self.get_word_embeddings(text)
        return np.mean(word_embeddings, axis=0)

swem = SWEM_Glove(embedding_dict, tokenizer)

In [143]:
# Word ngram vector
tr_vect = np.array([swem.average_pooling(text) for text in df_train['TITLE'].tolist()])
vl_vect = np.array([swem.average_pooling(text) for text in df_valid['TITLE'].tolist()])
ts_vect = np.array([swem.average_pooling(text) for text in df_test['TITLE'].tolist()])

In [144]:
model = lgb.LGBMClassifier()
model.fit(tr_vect, y_tr)

LGBMClassifier()

In [145]:
# 検証データを予測する
y_pred = model.predict_proba(vl_vect)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

accuracy = sum(y_vl == y_pred_max) / len(y_vl)
print('{:.4f}'.format(accuracy))
print('{:.4f}'.format(roc_auc_score(y_vl, y_pred, multi_class='ovo')))
print('{:.4f}'.format(log_loss(y_vl, y_pred)))

# 評価データを予測する
y_pred = model.predict_proba(ts_vect)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

accuracy = sum(y_te == y_pred_max) / len(y_te)
print('{:.4f}'.format(accuracy))
print('{:.4f}'.format(roc_auc_score(y_te, y_pred, multi_class='ovo')))
print('{:.4f}'.format(log_loss(y_te, y_pred)))

0.7560
0.8353
0.7070
0.7635
0.8543
0.6685


# FastText

In [3]:
from gensim.models import FastText
import fasttext

ModuleNotFoundError: No module named 'fasttext'

In [2]:
#model2 = FastText.load_fasttext_format('cc.en.300.bin')
FASTTEXT_MODEL_BIN = "fasttext_model.bin"
#this works
ft_model = FastText.load_model(FASTTEXT_MODEL_BIN)
ft_model.get_word_vector("additional")

AttributeError: type object 'FastText' has no attribute 'load_model'

In [ ]:
model2.wv['computer'].shape

In [ ]:
class SWEM_FastText():
    """
    Simple Word-Embeddingbased Models (SWEM)
    https://arxiv.org/abs/1805.09843v1
    """

    def __init__(self, dic, tokenizer, oov_initialize_range=(-0.01, 0.01)):
        self.tokenizer = tokenizer
        self.dic = dic
        self.embedding_dim = self.dic['a'].shape[0]
        self.oov_initialize_range = oov_initialize_range

        if self.oov_initialize_range[0] > self.oov_initialize_range[1]:
            raise ValueError("Specify valid initialize range: "
                             f"[{self.oov_initialize_range[0]}, {self.oov_initialize_range[1]}]")

    def get_word_embeddings(self, text):
        np.random.seed(abs(hash(text)) % (10 ** 8))

        vectors = []
        for word in text.split():
            if word in self.dic:
                vectors.append(self.dic[word])
            else:
                vectors.append(np.random.uniform(self.oov_initialize_range[0],
                                                 self.oov_initialize_range[1],
                                                 self.embedding_dim))
        return np.array(vectors)

    def average_pooling(self, text):
        word_embeddings = self.get_word_embeddings(text)
        return np.mean(word_embeddings, axis=0)

swem = SWEM_Glove(embedding_dict, tokenizer)